In [1]:
import json
import requests
import re
from bs4 import BeautifulSoup

In [2]:
def get_album_urls():
    """Returns list of URLs to every album"""
    page = requests.get('https://bobdylan.com/albums/')
    soup = BeautifulSoup(page.content, 'html.parser')
    return ([
        x.a['href']
        for x in soup.find_all('h3')])

In [3]:
album_urls = get_album_urls()

In [4]:
album_url = album_urls[11]

In [5]:
page = requests.get(album_url)
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
cover = soup.find(class_='cover').find('img')['src']
cover

'https://cdn.smehost.net/bobdylancom-uscolumbiaprod/wp-content/uploads/2011/11/5701829_179.jpg'

In [7]:
headline = soup.find(class_='headline').get_text(strip=True)
title = headline[:headline.find('(')].strip()
year = headline[headline.find("(")+1:headline.find(")")]
title, year

('The Best Of The Original Mono Recordings', '2010')

In [8]:
track_list = [
    x.get_text(strip='true').split('. ')[1]
    for x in soup.find_all(class_='track')]

In [9]:
def get_album_data(album_url):
    page = requests.get(album_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    cover_url = soup.find(class_='cover').find('img')['src']
    headline = (soup.find(class_='headline').get_text(strip=True))
    title = headline[:headline.find('(')].strip()
    year = headline[headline.find("(")+1:headline.find(")")]
    
    track_list = [
        x.get_text(strip='true').split('. ')[1]
        for x in  soup.find_all(class_='track')]
    
    return dict(
        title=title, year=year, cover_url=cover_url,
        track_list=track_list, url=album_url)

In [12]:
d1 = get_album_data(album_urls[11])
d2 = get_album_data(album_urls[12])

In [17]:
albums = [get_album_data(url) for url in album_urls]

In [18]:
with open('data/albums.json', 'w') as f:
    json.dump(albums, f)

In [21]:
# Load from my JSON
df = pd.read_json('data/albums.json')
df.set_index('title', inplace=True)

In [110]:
def get_setlist_urls(filter_url):
    page = requests.get(filter_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    tour_urls = soup.find(class_='tour-list').find_all('a', {'class', 'date'})
    return [x['href'] for x in tour_urls]

def get_filtered_tour_urls():
    page = requests.get('https://www.bobdylan.com/setlists')
    soup = BeautifulSoup(page.content, 'html.parser')
    
    return [
        x['href']
        for x in soup.find(class_='buy-options').find_all('a')]

def get_all_setlist_urls():
    all_setlist_urls = []
    filter_urls = get_filtered_tour_urls()
    for url in filter_urls:
        all_setlist_urls.extend(get_setlist_urls(url))
    return all_setlist_urls

In [1]:
# setlist_urls = get_all_setlist_urls()

In [111]:
page = requests.get(setlist_url)
soup = BeautifulSoup(page.content, 'html.parser')

In [112]:
date = soup.find(class_='date').get_text()
place = soup.find(class_='headline').get_text()
venue = soup.find(class_='venue').get_text()

setlist = [
    x.get_text()
    for x in soup.find(class_='set-list').find_all('a')]

In [138]:
def get_show_data(setlist_url):
    print(setlist_url)
    page = requests.get(setlist_url)
    soup = BeautifulSoup(page.content, 'html.parser')

    date = safe_parse(lambda: soup.find(class_='date').get_text())
    place = safe_parse(lambda: soup.find(class_='headline').get_text())
    venue = safe_parse(lambda: soup.find(class_='venue').get_text())
    _setlist = safe_parse(lambda: soup.find(class_='set-list').find_all(class_='title'))
    setlist = [x.get_text() for x in _setlist]
    
    print(dict(date=date, place=place, venue=venue, setlist=setlist))
    return dict(date=date, place=place, venue=venue, setlist=setlist)

In [139]:
def safe_parse(func):
    """Catches parsing errors"""
    try:
        return func()
    except AttributeError:
        print("  Couldn't parse!")
        return ''


In [142]:
with open('data/shows.json', 'w') as f:
    json.dump(show_data, f)

In [143]:
import pandas as pd

In [154]:
song_df = pd.read_json('data/songs.json').set_index('title')

In [156]:
album_df = pd.read_json('data/albums.json').set_index('title')